# IBM Coursera Capstone project The Battle of Neighborhoods

## Business problem

### Background

Pakistan is a captivating place due to its geographical significance and Lahore is one of most popular city to live and start a business. In last few years world has realized its importance and start investing in it. Since 2010 many international companies from different sectors like Automobile, Gas & Oil, Agriculture, fashion and consturction establishing their businesses in Pakistan. CPEC a multi-billion project with China is about to complete and after that world will be attract towards Pakistan and everyone will be in hurry to invest at the right time and at the right place.

### Business Problem

As we can see this is the right time to analyze the trends and invest your money so you will get the best out of your desision using machine learning approches. Here we will work on real state sector as this is always considered one of the most profit generated area. We will see which place is best to buy according to your need. For example if someone wants to buy a place at non commertial area than where should he invest his minimum money to possess maximum land. Or if someone wants to start a business than definitly he wants a place which is surrounded by many people having interest related to their business.

## Data Collection

Data is the base of every experiment. So, we will use Zameen.com dataset which include property type, price, location, area, bathrooms, bedrooms and some other features. You can download the dataset from kaggle 'https://www.kaggle.com/huzzefakhan/zameencom-property-data-pakistan'. The dataset have many columns and some of them are not required in our analysis. We will remove and manipulate our data according to our need.

For location, tips and Venue services we will use Foursquare API and find the trends in the specified area.

## Methodology section

The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

#### 1. Collect Inspection Data

After importing the necessary libraries, we download the data from the Kaggle website as follows:

In [1]:
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.distance import vincenty

# import k-means from clustering stage
from sklearn.cluster import KMeans

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
# Reading data from file
property_df = pd.read_csv('Zameen Property Data.csv')

#### 2. Explore and Understand Data

As we have loaded our dataset. Its time to explore it.

In [3]:
property_df.head()

,property_id,location_id,page_url,property_type,price,location,city,province_name,latitude,longitude,baths,area,purpose,bedrooms,date_added,agency,agent
0,237062,3325,https://www.zameen.com/Property/g_10_g_10_2_gr...,Flat,10000000,G-10,Islamabad,Islamabad Capital,33.679890,73.012640,2,4 Marla,For Sale,2,02-04-2019,NaN,NaN
1,346905,3236,https://www.zameen.com/Property/e_11_2_service...,Flat,6900000,E-11,Islamabad,Islamabad Capital,33.700993,72.971492,3,5.6 Marla,For Sale,3,05-04-2019,NaN,NaN
2,386513,764,https://www.zameen.com/Property/islamabad_g_15...,House,16500000,G-15,Islamabad,Islamabad Capital,33.631486,72.926559,6,8 Marla,For Sale,5,07-17-2019,NaN,NaN
3,656161,340,https://www.zameen.com/Property/islamabad_bani...,House,43500000,Bani Gala,Islamabad,Islamabad Capital,33.707573,73.151199,4,2 Kanal,For Sale,4,04-05-2019,NaN,NaN
4,841645,3226,https://www.zameen.com/Property/dha_valley_dha...,House,7000000,DHA Defence,Islamabad,Islamabad Capital,33.492591,73.301339,3,8 Marla,For Sale,3,07-10-2019,Easy Property,Muhammad Junaid Ceo Muhammad Shahid Director


In [4]:
property_df.shape

(168446, 17)

#### 3. Data preparation and preprocessing

We can see we have many columns that are not necessary in our experiment. So, here we will drop those columns.

In [5]:
property_df.drop(['property_id', 'location_id', 'page_url', 'date_added', 'agency', 'agent'], axis = 1, inplace = True)
property_df.head()

,property_type,price,location,city,province_name,latitude,longitude,baths,area,purpose,bedrooms
0,Flat,10000000,G-10,Islamabad,Islamabad Capital,33.679890,73.012640,2,4 Marla,For Sale,2
1,Flat,6900000,E-11,Islamabad,Islamabad Capital,33.700993,72.971492,3,5.6 Marla,For Sale,3
2,House,16500000,G-15,Islamabad,Islamabad Capital,33.631486,72.926559,6,8 Marla,For Sale,5
3,House,43500000,Bani Gala,Islamabad,Islamabad Capital,33.707573,73.151199,4,2 Kanal,For Sale,4
4,House,7000000,DHA Defence,Islamabad,Islamabad Capital,33.492591,73.301339,3,8 Marla,For Sale,3


In [6]:
property_df.describe()

,price,latitude,longitude,baths,bedrooms
count,1.684460e+05,168446.000000,168446.000000,168446.000000,168446.000000
mean,1.776576e+07,29.859519,71.239804,2.874227,3.179422
std,3.531003e+07,3.807870,3.133042,2.463400,1.971401
min,0.000000e+00,11.052446,25.906027,0.000000,0.000000
25%,1.750000e+05,24.948536,67.130363,0.000000,2.000000
50%,8.500000e+06,31.459784,73.056182,3.000000,3.000000
75%,1.950000e+07,33.560887,73.259870,4.000000,4.000000
max,2.000000e+09,73.184088,80.161430,403.000000,68.000000


We can see outlier in our data by analyzing minimum price. Price cannot be zero to buy a place so we set a threshold and only consider data where Property type is House and price of house is at least Rs.1000000 PKR for sale in Lahore.

In [7]:
a = property_df[(property_df["property_type"] == "House") & (property_df['price'] > 999999) & 
                (property_df['purpose'] == 'For Sale') & (property_df['city'] == 'Lahore')]
a.shape

(39324, 11)

In [8]:
# Make a list of street names in Lahore
streets = a['location'].unique().tolist()

property_lahore_price = a.groupby(['location'])['price', 'latitude', 'longitude'].mean().reset_index()

# Give meaningful names to the columns
property_lahore_price.columns = ['Street', 'Avg_Price', 'Latitude', 'Longitude']
property_lahore_price

,Street,Avg_Price,Latitude,Longitude
0,AR Cottages,1.150000e+07,31.495734,74.426671
1,AWT Army Welfare Trust,1.226667e+07,31.356532,74.210425
2,Aabpara Coop Housing Society,3.540000e+07,31.443685,74.205662
3,Aashiana Road,4.828713e+06,31.429120,74.378989
4,Abdali Road,2.130000e+07,31.568747,74.292991
...,...,...,...,...
600,Zaitoon - New Lahore City,7.702812e+06,31.347894,74.142697
601,Zaitoon Colony,1.120000e+07,31.585410,74.409010
602,Zaman Colony,1.693000e+07,31.495177,74.371390
603,Zaman Park,7.400000e+07,31.551733,74.352636


In [9]:
for index, item in property_lahore_price.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 0
item: Street       AR Cottages
Avg_Price       1.15e+07
Latitude         31.4957
Longitude        74.4267
Name: 0, dtype: object
item.Street only: AR Cottages
index: 1
item: Street       AWT Army Welfare Trust
Avg_Price               1.22667e+07
Latitude                    31.3565
Longitude                   74.2104
Name: 1, dtype: object
item.Street only: AWT Army Welfare Trust
index: 2
item: Street       Aabpara Coop Housing Society
Avg_Price                        3.54e+07
Latitude                          31.4437
Longitude                         74.2057
Name: 2, dtype: object
item.Street only: Aabpara Coop Housing Society
index: 3
item: Street       Aashiana Road
Avg_Price      4.82871e+06
Latitude           31.4291
Longitude           74.379
Name: 3, dtype: object
item.Street only: Aashiana Road
index: 4
item: Street       Abdali Road
Avg_Price       2.13e+07
Latitude         31.5687
Longitude         74.293
Name: 4, dtype: object
item.Street only: Abdali Road
index: 5
i

Name: 176, dtype: object
item.Street only: Ferozepur City
index: 177
item: Street       Ferozepur Road
Avg_Price       6.12701e+06
Latitude            31.4536
Longitude           74.3496
Name: 177, dtype: object
item.Street only: Ferozepur Road
index: 178
item: Street       Firdous Colony
Avg_Price          1.15e+07
Latitude            31.5448
Longitude           74.2805
Name: 178, dtype: object
item.Street only: Firdous Colony
index: 179
item: Street       Formanites Housing Scheme
Avg_Price                  1.00974e+07
Latitude                       31.4471
Longitude                      74.4136
Name: 179, dtype: object
item.Street only: Formanites Housing Scheme
index: 180
item: Street       Fort Villas
Avg_Price     2.0525e+07
Latitude         31.4861
Longitude        74.3781
Name: 180, dtype: object
item.Street only: Fort Villas
index: 181
item: Street              GOR
Avg_Price    1.9425e+08
Latitude        31.5434
Longitude       74.3442
Name: 181, dtype: object
item.Street only

Name: 315, dtype: object
item.Street only: Lahore Cantt Coop Housing
index: 316
item: Street       Lahore Garden Housing Scheme
Avg_Price                     7.03021e+06
Latitude                          31.5906
Longitude                         74.2082
Name: 316, dtype: object
item.Street only: Lahore Garden Housing Scheme
index: 317
item: Street       Lahore Medical Housing Society
Avg_Price                       1.03684e+07
Latitude                            31.5819
Longitude                           74.4636
Name: 317, dtype: object
item.Street only: Lahore Medical Housing Society
index: 318
item: Street       Lahore Motorway City
Avg_Price             5.58065e+06
Latitude                  31.6435
Longitude                 74.2099
Name: 318, dtype: object
item.Street only: Lahore Motorway City
index: 319
item: Street       Lahore Press Club Housing Scheme
Avg_Price                         1.76111e+07
Latitude                              31.5799
Longitude                          

index: 460
item: Street       Rasool Park
Avg_Price    1.55294e+07
Latitude         31.5409
Longitude        74.3106
Name: 460, dtype: object
item.Street only: Rasool Park
index: 461
item: Street       Ravi Park
Avg_Price    1.448e+07
Latitude       31.5953
Longitude      74.3065
Name: 461, dtype: object
item.Street only: Ravi Park
index: 462
item: Street         Ravi Road
Avg_Price    1.89286e+07
Latitude         31.5941
Longitude        74.3027
Name: 462, dtype: object
item.Street only: Ravi Road
index: 463
item: Street       Real Cottages
Avg_Price            1e+07
Latitude            31.488
Longitude          74.4088
Name: 463, dtype: object
item.Street only: Real Cottages
index: 464
item: Street       Rehan Garden
Avg_Price       1.036e+07
Latitude          31.3453
Longitude          74.379
Name: 464, dtype: object
item.Street only: Rehan Garden
index: 465
item: Street       Rehman City - Phase 4
Avg_Price                  4.4e+06
Latitude                   31.5794
Longitude      

Name: 576, dtype: object
item.Street only: UET Housing Society
index: 577
item: Street       UMT Society
Avg_Price      9.825e+06
Latitude         31.4523
Longitude        74.2939
Name: 577, dtype: object
item.Street only: UMT Society
index: 578
item: Street        Upper Mall
Avg_Price    6.11962e+07
Latitude         31.5421
Longitude        74.3559
Name: 578, dtype: object
item.Street only: Upper Mall
index: 579
item: Street       Urdu Bazar
Avg_Price         3e+07
Latitude        31.5748
Longitude       74.3096
Name: 579, dtype: object
item.Street only: Urdu Bazar
index: 580
item: Street       Valencia Housing Society
Avg_Price                 3.16536e+07
Latitude                      31.4044
Longitude                     74.2571
Name: 580, dtype: object
item.Street only: Valencia Housing Society
index: 581
item: Street       Venus Housing Scheme
Avg_Price               1.396e+07
Latitude                  31.4355
Longitude                 74.3488
Name: 581, dtype: object
item.Street 

In [10]:
address = 'Lahore, Pakistan'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lahore City are {}, {}.'.format(latitude, longitude))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of Lahore City are 31.5656079, 74.3141775.


In [11]:
# create map of Lahore using latitude and longitude values
map_lahore = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(property_lahore_price['Latitude'], property_lahore_price['Longitude'], property_lahore_price['Avg_Price'], property_lahore_price['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lahore)  
    
map_lahore

In [12]:
#Define Foursquare Credentials and Version

CLIENT_ID = '3MV4HG55L3OSKZ2FIQ2WXP25CKDF41TUL30CTHH5KOK2URYY' # Foursquare ID
CLIENT_SECRET = 'UH02LI4NCKO3MFHQR0FZIXBDFBY1YFJZJCAFJT1DTYGFYJ1O' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3MV4HG55L3OSKZ2FIQ2WXP25CKDF41TUL30CTHH5KOK2URYY
CLIENT_SECRET:UH02LI4NCKO3MFHQR0FZIXBDFBY1YFJZJCAFJT1DTYGFYJ1O


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

#### 4. Modeling

After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in Lahore and is accurate.

In [13]:

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=property_lahore_price['Street'],
                                   latitudes=property_lahore_price['Latitude'],
                                   longitudes=property_lahore_price['Longitude']
                                  )

AR Cottages
AWT Army Welfare Trust
Aabpara Coop Housing Society
Aashiana Road
Abdali Road
Abdalians Cooperative Housing Society
Abdul Sattar Edhi Road
Abdullah Town
Abid Market
Abu Alkhair Road
Acro Green - Bankers Avenue
Adan Villas
Afzal Park
Agrics Town
Ahbab Colony
Ahlu Road
Ahmad Avenue
Ahmad Housing Society
Airline Housing Society
Airport Road
Aitchison Society
Al Faisal Town
Al Fajar Society
Al Hafeez Gardens
Al Hamad Colony
Al Hamra Town
Al Haram Garden
Al Jalil Garden
Al Jannat Housing Scheme
Al Kareem Garden
Al Madina Avenue
Al Noor Town
Al Raheem Town
Al Rahim Homes
Al Rehman Garden
Al-Amin Housing Society
Al-Falah Cooperative Housing Society
Al-Hafeez Garden Road
Al-Hafiz Town
Al-Hamad Colony (AIT)
Al-Hamd Garden
Al-Hamd Park
Al-Jannat Housing Society - Kahna
Al-Kabir Town
Al-Kareem Premier Housing Scheme
Al-Raheem Garden
Al-Raziq Garden
Al-Wahab Garden - Phase 1
Alfalah Town
Ali Alam Garden
Ali Town
Ali View Garden
Ali View Park
Alia Town
Allama Iqbal Intl Airport
Allama I

Shamkay Bhattian
Shanghai Road
Sharaqpur Road
Sheikhupura Road
Sher Shah Road
Shera Kot
Sheraz Town
Shershah Colony - Ichra
Shershah Colony - Raiwind Road
Sherwani Town Housing Scheme
Shiraz Villas
Shoukat Town
Shuja Road
Siddiqia Colony
Siddiqia Society (College Road)
Sikandar Road
Singhpura
Sitara Colony
Sodiwal
Sozo Town
State Life Housing Society
Sue-e-Asal
Sufiabad
Sui Gas Employees Cooperative Housing Society
Sui Gas Housing Society
Sukh Chayn Gardens
Sultan Colony
Sultan Pura
Sultan Town
Sundar
Sunderdas Road
Sunflower Housing Society
Sunfort Gardens
Sunny Park
Super Town
Supreme Villas
Swami Nagar
T & T Aabpara Housing Society
TECH Society
TIP Housing Society
Taj Bagh Scheme
Tajpura
Tariq Gardens
Tariq Ismail Road
Tarogill
Temple Road
Thokar Niaz Baig
Toheed Park
Township
Tricon Village
Tulspura
UBL Housing Society
UET Housing Society
UMT Society
Upper Mall
Urdu Bazar
Valencia Housing Society
Venus Housing Scheme
Vital Homes Housing Scheme
Wafaqi Colony
Wahdat Colony
Wahdat Roa

In [15]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
AR Cottages,2,2,2,2,2,2
Abdali Road,4,4,4,4,4,4
Abdalians Cooperative Housing Society,3,3,3,3,3,3
Abid Market,4,4,4,4,4,4
Afzal Park,1,1,1,1,1,1
...,...,...,...,...,...,...
Wyeth Employees Coop Housing,1,1,1,1,1,1
Zaheer Villas,1,1,1,1,1,1
Zaitoon Colony,2,2,2,2,2,2


In [16]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 176 uniques categories.


In [17]:
location_venues.shape

(1193, 7)

In [18]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Asian Restaurant,...,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Video Store,Volleyball Court,Warehouse Store,Water Park,Zoo
0,AR Cottages,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AR Cottages,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Abdali Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abdali Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abdali Road,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
venues_onehot.shape

(1193, 177)

In [20]:
lahore_grouped = venues_onehot.groupby('Street').mean().reset_index()
lahore_grouped.head()

,Street,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Asian Restaurant,...,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Video Store,Volleyball Court,Warehouse Store,Water Park,Zoo
0,AR Cottages,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abdali Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abdalians Cooperative Housing Society,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abid Market,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Afzal Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
lahore_grouped.shape

(370, 177)

In [22]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in lahore_grouped['Street']:
    print("----"+hood+"----")
    temp = lahore_grouped[lahore_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AR Cottages----
               venue  freq
0   Asian Restaurant   0.5
1               Café   0.5
2  Accessories Store   0.0
3       Neighborhood   0.0
4              Motel   0.0


----Abdali Road----
                 venue  freq
0          Pizza Place  0.25
1  Fried Chicken Joint  0.25
2            Juice Bar  0.25
3            BBQ Joint  0.25
4         Neighborhood  0.00


----Abdalians Cooperative Housing Society----
                venue  freq
0         Pizza Place  0.33
1  Italian Restaurant  0.33
2    Asian Restaurant  0.33
3        Neighborhood  0.00
4               Motel  0.00


----Abid Market----
                  venue  freq
0             BBQ Joint  0.50
1  Pakistani Restaurant  0.25
2                Market  0.25
3          Neighborhood  0.00
4                 Motel  0.00


----Afzal Park----
               venue  freq
0        Bus Station   1.0
1  Accessories Store   0.0
2           Platform   0.0
3              Motel   0.0
4    Motorcycle Shop   0.0


----Ahmad Avenue---

               venue  freq
0           Tea Room   0.5
1         Restaurant   0.5
2  Accessories Store   0.0
3      National Park   0.0
4             Mosque   0.0


----Batapur----
               venue  freq
0      Shopping Mall   1.0
1  Accessories Store   0.0
2        Pizza Place   0.0
3             Mosque   0.0
4              Motel   0.0


----Beadon Road----
              venue  freq
0    Ice Cream Shop  0.09
1       Pizza Place  0.09
2  Department Store  0.09
3    Clothing Store  0.09
4    Hardware Store  0.09


----Begampura----
                 venue  freq
0            BBQ Joint   1.0
1  Monument / Landmark   0.0
2                Motel   0.0
3      Motorcycle Shop   0.0
4             Mountain   0.0


----Bhaini Road----
                venue  freq
0  Seafood Restaurant   1.0
1   Accessories Store   0.0
2         Pizza Place   0.0
3              Mosque   0.0
4               Motel   0.0


----Bhatta Chowk----
                     venue  freq
0           Breakfast Spot   0.5
1  Nort

                  venue  freq
0            Steakhouse   1.0
1     Accessories Store   0.0
2  Mongolian Restaurant   0.0
3                Mosque   0.0
4                 Motel   0.0


----Eden----
               venue  freq
0               Park   0.5
1         Restaurant   0.5
2  Accessories Store   0.0
3      National Park   0.0
4             Mosque   0.0


----Eden Avenue Extension----
                  venue  freq
0           Pizza Place  0.17
1                  Park  0.17
2  Fast Food Restaurant  0.17
3      Asian Restaurant  0.17
4         Shopping Mall  0.17


----Eden Villas----
                  venue  freq
0      Department Store  0.11
1                Bakery  0.11
2            Shoe Store  0.11
3         Shopping Mall  0.11
4  Pakistani Restaurant  0.11


----Education Town----
                     venue  freq
0                BBQ Joint   0.5
1  Fruit & Vegetable Store   0.5
2             Neighborhood   0.0
3                    Motel   0.0
4          Motorcycle Shop   0.0


----

4    Motorcycle Shop   0.0


----Hall Road----
                  venue  freq
0           Pizza Place  0.12
1        Ice Cream Shop  0.12
2        Hardware Store  0.12
3        Clothing Store  0.12
4  Pakistani Restaurant  0.12


----Hameedpura----
               venue  freq
0        Pizza Place   1.0
1  Afghan Restaurant   0.0
2             Mosque   0.0
3              Motel   0.0
4    Motorcycle Shop   0.0


----Hamza Town----
                     venue  freq
0           Clothing Store   0.5
1               Print Shop   0.5
2        Accessories Store   0.0
3  North Indian Restaurant   0.0
4          Motorcycle Shop   0.0


----Hanif Park Harbanspura----
               venue  freq
0        Bus Station   1.0
1  Accessories Store   0.0
2           Platform   0.0
3              Motel   0.0
4    Motorcycle Shop   0.0


----Harbanspura----
               venue  freq
0        Bus Station   1.0
1  Accessories Store   0.0
2           Platform   0.0
3              Motel   0.0
4    Motorcycle Sho

                  venue  freq
0  Fast Food Restaurant  0.27
1             Juice Bar  0.18
2     Afghan Restaurant  0.09
3           Pizza Place  0.09
4        Breakfast Spot  0.09


----Lahore Press Club Housing Scheme----
                  venue  freq
0  Brazilian Restaurant   1.0
1     Accessories Store   0.0
2              Platform   0.0
3                 Motel   0.0
4       Motorcycle Shop   0.0


----Lahore, Punjab----
               venue  freq
0              Motel  0.67
1   Asian Restaurant  0.33
2  Accessories Store  0.00
3       Neighborhood  0.00
4    Motorcycle Shop  0.00


----Lake City----
               venue  freq
0          Multiplex   0.5
1       Burger Joint   0.5
2  Accessories Store   0.0
3           Platform   0.0
4              Motel   0.0


----Lakshmi Chowk----
                  venue  freq
0      Asian Restaurant  0.50
1  Pakistani Restaurant  0.17
2          Dessert Shop  0.17
3            Restaurant  0.17
4     Accessories Store  0.00


----Lalazaar Garden---

4    Motorcycle Shop   0.0


----Nasheman-e-Iqbal----
               venue  freq
0             Bakery  0.67
1               Café  0.33
2  Accessories Store  0.00
3       Neighborhood  0.00
4              Motel  0.00


----Nasirabad----
               venue  freq
0    Badminton Court  0.33
1       Soccer Field  0.33
2       Dessert Shop  0.33
3  Accessories Store  0.00
4              Motel  0.00


----National Town----
               venue  freq
0       Burger Joint   1.0
1  Accessories Store   0.0
2           Platform   0.0
3              Motel   0.0
4    Motorcycle Shop   0.0


----Nayyab Villas----
                    venue  freq
0             Men's Store  0.33
1  Furniture / Home Store  0.33
2                  Bakery  0.33
3       Accessories Store  0.00
4     Monument / Landmark  0.00


----Neela Gumbad----
                  venue  freq
0        History Museum   0.2
1         Historic Site   0.2
2    Travel & Transport   0.2
3  Pakistani Restaurant   0.2
4             Bookstore   0

4            Motel  0.00


----Rang Mahal----
               venue  freq
0  Accessories Store  0.17
1      Historic Site  0.17
2     Clothing Store  0.17
3     Breakfast Spot  0.17
4      Jewelry Store  0.17


----Rasool Park----
                    venue  freq
0                    Park   0.5
1  Furniture / Home Store   0.5
2       Accessories Store   0.0
3           National Park   0.0
4                   Motel   0.0


----Ravi Park----
                 venue  freq
0  Monument / Landmark   0.5
1        National Park   0.5
2    Afghan Restaurant   0.0
3               Mosque   0.0
4                Motel   0.0


----Ravi Road----
                  venue  freq
0                Bakery  0.17
1       Warehouse Store  0.17
2          Burger Joint  0.17
3  Gym / Fitness Center  0.17
4              Tea Room  0.17


----Real Cottages----
                     venue  freq
0                 Pharmacy  0.25
1           Breakfast Spot  0.25
2  North Indian Restaurant  0.25
3                   Bakery  

                venue  freq
0  Italian Restaurant   0.5
1    Asian Restaurant   0.5
2   Accessories Store   0.0
3        Neighborhood   0.0
4               Motel   0.0


----Sheraz Town----
               venue  freq
0         Restaurant   0.5
1             Bakery   0.5
2  Accessories Store   0.0
3       Neighborhood   0.0
4              Motel   0.0


----Shershah Colony - Ichra----
                 venue  freq
0     Asian Restaurant  0.33
1               Market  0.17
2  Fried Chicken Joint  0.17
3         Dessert Shop  0.17
4          Bus Station  0.17


----Shiraz Villas----
               venue  freq
0      Jewelry Store  0.25
1   Department Store  0.25
2               Café  0.25
3             Bakery  0.25
4  Accessories Store  0.00


----Shoukat Town----
               venue  freq
0        Dairy Store   1.0
1  Accessories Store   0.0
2       Neighborhood   0.0
3              Motel   0.0
4    Motorcycle Shop   0.0


----Shuja Road----
                  venue  freq
0  Fast Food Resta

               venue  freq
0  Convenience Store   1.0
1  Accessories Store   0.0
2       Neighborhood   0.0
3              Motel   0.0
4    Motorcycle Shop   0.0


----Zaitoon Colony----
               venue  freq
0  Fish & Chips Shop   0.5
1       Cupcake Shop   0.5
2  Accessories Store   0.0
3      National Park   0.0
4              Motel   0.0


----Zaman Colony----
                   venue  freq
0    Fried Chicken Joint  0.25
1   Pakistani Restaurant  0.25
2         Breakfast Spot  0.25
3  Outdoors & Recreation  0.25
4      Accessories Store  0.00


----Zubaida Park----
                  venue  freq
0  Fast Food Restaurant   0.5
1  Pakistani Restaurant   0.5
2     Accessories Store   0.0
3         National Park   0.0
4                 Motel   0.0




In [23]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [25]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = lahore_grouped['Street']

for ind in np.arange(lahore_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(lahore_grouped.iloc[ind, :], num_top_venues)

In [26]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AR Cottages,Café,Asian Restaurant,Zoo,Doctor's Office,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
1,Abdali Road,Pizza Place,Juice Bar,Fried Chicken Joint,BBQ Joint,Convenience Store,Coworking Space,Field,Fast Food Restaurant,Farmers Market,Farm
2,Abdalians Cooperative Housing Society,Pizza Place,Italian Restaurant,Asian Restaurant,Diner,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
3,Abid Market,BBQ Joint,Pakistani Restaurant,Market,Zoo,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
4,Afzal Park,Bus Station,Coffee Shop,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop


In [27]:
venues_sorted.shape

(370, 11)

In [28]:
lahore_grouped.shape

(370, 177)

In [29]:
lahore_grouped = property_lahore_price

After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in Lahore, we could begin by clustering properties by venues/facilities/amenities nearby.

In [30]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

lahore_grouped_clustering = lahore_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lahore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([4, 4, 0, 4, 1, 0, 0, 4, 0, 4, 1, 4, 4, 0, 4, 4, 4, 4, 1, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 1, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4])

In [31]:
#Dataframe to include Clusters

lahore_grouped_clustering = property_lahore_price
lahore_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
0,AR Cottages,1.150000e+07,31.495734,74.426671
1,AWT Army Welfare Trust,1.226667e+07,31.356532,74.210425
2,Aabpara Coop Housing Society,3.540000e+07,31.443685,74.205662
3,Aashiana Road,4.828713e+06,31.429120,74.378989
4,Abdali Road,2.130000e+07,31.568747,74.292991


In [32]:
lahore_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [33]:
# add clustering labels
lahore_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge lahore_grouped with lahore_data to add latitude/longitude for each neighborhood
lahore_grouped_clustering = lahore_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

lahore_grouped_clustering.head(5) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AR Cottages,1.150000e+07,31.495734,74.426671,4,Café,Asian Restaurant,Zoo,Doctor's Office,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
1,AWT Army Welfare Trust,1.226667e+07,31.356532,74.210425,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aabpara Coop Housing Society,3.540000e+07,31.443685,74.205662,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aashiana Road,4.828713e+06,31.429120,74.378989,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abdali Road,2.130000e+07,31.568747,74.292991,1,Pizza Place,Juice Bar,Fried Chicken Joint,BBQ Joint,Convenience Store,Coworking Space,Field,Fast Food Restaurant,Farmers Market,Farm


In [34]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lahore_grouped_clustering['Latitude'], lahore_grouped_clustering['Longitude'], lahore_grouped_clustering['Street'], lahore_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analyzing Clusters

#### Cluster 1 

In [35]:
lahore_grouped_clustering.loc[lahore_grouped_clustering['Cluster Labels'] == 0, lahore_grouped_clustering.columns[[1] + list(range(5, lahore_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,3.540000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.095821e+07,Pizza Place,Italian Restaurant,Asian Restaurant,Diner,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
6,4.212500e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4.145000e+07,BBQ Joint,Pakistani Restaurant,Market,Zoo,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
13,3.325000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 2 

In [36]:
lahore_grouped_clustering.loc[lahore_grouped_clustering['Cluster Labels'] == 1, lahore_grouped_clustering.columns[[1] + list(range(5, lahore_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,2.130000e+07,Pizza Place,Juice Bar,Fried Chicken Joint,BBQ Joint,Convenience Store,Coworking Space,Field,Fast Food Restaurant,Farmers Market,Farm
10,1.692308e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,2.604878e+07,Fast Food Restaurant,Middle Eastern Restaurant,Snack Place,Zoo,Doctor's Office,Field,Farmers Market,Farm,Electronics Store,Dry Cleaner
31,1.603333e+07,Furniture / Home Store,Grocery Store,Supermarket,Zoo,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
33,1.466667e+07,Pharmacy,Ice Cream Shop,Italian Restaurant,Asian Restaurant,Zoo,Diner,Fast Food Restaurant,Farmers Market,Farm,Electronics Store


#### Cluster 3

In [37]:
lahore_grouped_clustering.loc[lahore_grouped_clustering['Cluster Labels'] == 2, lahore_grouped_clustering.columns[[1] + list(range(5, lahore_grouped_clustering.shape[1]))]]

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
341,448700000.0,Hotel,Zoo,Indian Restaurant,Breakfast Spot,Pakistani Restaurant,Mediterranean Restaurant,Resort,Bagel Shop,Multiplex,Japanese Restaurant


#### Cluster 4

In [38]:
lahore_grouped_clustering.loc[lahore_grouped_clustering['Cluster Labels'] == 3, lahore_grouped_clustering.columns[[1] + list(range(5, lahore_grouped_clustering.shape[1]))]]

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
107,1.050000e+08,Auto Workshop,Bakery,Doctor's Office,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
134,8.000000e+07,Park,Zoo,Diner,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
181,1.942500e+08,Social Club,Zoo,Cocktail Bar,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner,Donut Shop
198,8.103333e+07,Clothing Store,Café,Zoo,Diner,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
201,1.056250e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,1.048186e+08,Café,Fast Food Restaurant,Shopping Mall,Pakistani Restaurant,BBQ Joint,Asian Restaurant,Boutique,Bookstore,Mobile Phone Shop,Burger Joint
315,1.000000e+08,Fast Food Restaurant,Juice Bar,Pakistani Restaurant,Afghan Restaurant,Pizza Place,Breakfast Spot,Toy / Game Store,Bakery,Farmers Market,Farm
324,9.000000e+07,Asian Restaurant,Dessert Shop,Pakistani Restaurant,Restaurant,Diner,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
329,1.100000e+08,History Museum,Market,Food,Food Court,Diner,Coworking Space,Cricket Ground,Comfort Food Restaurant,Fast Food Restaurant,Construction & Landscaping
352,1.300000e+08,Asian Restaurant,Shopping Mall,Ice Cream Shop,Zoo,Doctor's Office,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store


#### Cluster 5

In [39]:
lahore_grouped_clustering.loc[lahore_grouped_clustering['Cluster Labels'] == 4, lahore_grouped_clustering.columns[[1] + list(range(5, lahore_grouped_clustering.shape[1]))]]

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1.150000e+07,Café,Asian Restaurant,Zoo,Doctor's Office,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store
1,1.226667e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.828713e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3.000000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.500000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
597,4.550000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,4.971429e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,7.889474e+06,Convenience Store,Zoo,Doctor's Office,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Dry Cleaner
600,7.702812e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Results and Discussion

Finally we are done with our experiment and now it is the time to discuss our findings. We split our data into 5 big clusters. By analyzing that we can say that **Cluster 1** is good place to live or to start food business for those who want themselves surrounded by Food and cafes. You need average price from 29,900,000 to 67,500,000 PKR to buy 2 Marla to 6 Kanal place. 

**Cluster 2** will be the best option for those who want entertainment(Cinemas, Multiplex, Art Gellary etc) as well as cannot compromise on shopping. You do not have to travel a lot to buy household stuff as you are surrounded by many supermarkets, stores, Book Shops etc. **Cluster 3** (Mall Road) is best and trendiest place for Hotel and Resort Businesses. Pakistan's most famous hotels are located on Mall Road e.g Pearl Continental, Avari Hotel and Flatties Hotel are the most renowned ones. So if you have a lot of money in other words avergely 450,000,000 PKR than you are all set to battle against these top notch hotel brands. 

**Cluater 4 and 5** are the places which are usually the side areas and has less population and may attract those people who want to live in city without getting rid of luxury and also want to stick with nature but do not like the city traffic and noise polution. 

# Conclusion

To solve this business problem, we clustered Lahore neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on Lahore properties and the relative price paid data were extracted from kaggle. Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on Lahore properties and the relative price paid data from the kaggle and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in Lahore and is accurate.